<a href="https://colab.research.google.com/github/bigboom1253/e_commerce_recommendation_system_development/blob/master/Data_preprocessing/Review_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from wordcloud import WordCloud
from collections import Counter
import datetime

plt.rc('font', family='NanumBarunGothic')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path = "/content/drive/My Drive/최강1조-추천 시스템/데이터 수집/"


prod = pd.read_json(path + "all_prod_data.json")
review = pd.read_json(path + "all_review_data.json")
# satisfaction = pd.read_json(path + "all_satisfaction_data.json")

In [3]:
train = pd.read_json("/content/drive/My Drive/최강1조-추천 시스템/데이터 전처리/train_reveiew_data.json")
test = pd.read_json("/content/drive/My Drive/최강1조-추천 시스템/데이터 전처리/test_reveiew_data.json")

In [6]:
len(train), len(test)

(125471, 125474)

# Review

In [6]:
# from datetime import datetime

# def to_datetime(x):
#     if x != "": return datetime.strptime(x[:10], "%Y-%m-%d")

# review["review_date"] = review.review_date.apply(lambda x: to_datetime(x)) # 전처리 - datetime 형태 변환
# review = review.merge(prod[['prod_idx', 'category_num', 'title', 'small_category']]) # 전처리 - 카테고리 넘버 merge 시켜줌

In [7]:
# review = review.dropna()

In [8]:
word = review[['memberSrl', 'content', 'prod_satis', 'ship_satis']]
word

,memberSrl,content,prod_satis,ship_satis
0,50349361,생각보다 조금 얇지만 맛있게 먹을게요,4,5
1,57382622,배송빠르고 맛도 괜찮아요.,4,5
2,7946305,달지도 않고 퍽퍽...,1,3
3,26446505,맛있어요 ㅋㅋㅋ 저렴해요,5,4
4,16284557,싱싱하고 크기도 좋아요 근데 맛이 좀 덜들었네요좀난뒀다 먹어야겠어요,5,5
...,...,...,...,...
1754921,8362913,최고포장 아 근데 진짜 많이,5,5
1754922,21157725,진공포장에 아이스박스에 안전하게 배달잘 왔어요,5,5
1754923,56932594,좋아요 싸게 잘샀어요 좋아요,4,5
1754924,29006409,제품 1개씩당 아이스팩을 같이묶어서 신선하게 잘받았네요,4,5


In [9]:
word['length'] = word.content.apply(len)
word

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,memberSrl,content,prod_satis,ship_satis,length
0,50349361,생각보다 조금 얇지만 맛있게 먹을게요,4,5,20
1,57382622,배송빠르고 맛도 괜찮아요.,4,5,15
2,7946305,달지도 않고 퍽퍽...,1,3,13
3,26446505,맛있어요 ㅋㅋㅋ 저렴해요,5,4,13
4,16284557,싱싱하고 크기도 좋아요 근데 맛이 좀 덜들었네요좀난뒀다 먹어야겠어요,5,5,37
...,...,...,...,...,...
1754921,8362913,최고포장 아 근데 진짜 많이,5,5,16
1754922,21157725,진공포장에 아이스박스에 안전하게 배달잘 왔어요,5,5,25
1754923,56932594,좋아요 싸게 잘샀어요 좋아요,4,5,16
1754924,29006409,제품 1개씩당 아이스팩을 같이묶어서 신선하게 잘받았네요,4,5,30


In [7]:
def clean(x):
    text = re.sub('[-=_+,#\?;:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》☆★×♡♥·ᆢ\/]', '', x) # 특수문자 제거
    text = re.sub('[a-zA-Z]', '', text) # 영어 제거
    text = re.sub('[ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ]', '', text) # 모음 제거
    text = re.sub('[ㄱ-ㅎ]*', '', text) # 자음 반복 제거
    text = re.sub('[0-9]{3,}', '', text) # 숫자 3번 이상 나오면 제거
    return text

In [12]:
word['clean'] = word.content.apply(clean)
word['length_clean'] = word.clean.apply(len)
word

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,memberSrl,content,prod_satis,ship_satis,length,clean,length_clean
0,50349361,생각보다 조금 얇지만 맛있게 먹을게요,4,5,20,생각보다 조금 얇지만 맛있게 먹을게요,20
1,57382622,배송빠르고 맛도 괜찮아요.,4,5,15,배송빠르고 맛도 괜찮아요,14
2,7946305,달지도 않고 퍽퍽...,1,3,13,달지도 않고 퍽퍽,10
3,26446505,맛있어요 ㅋㅋㅋ 저렴해요,5,4,13,맛있어요 저렴해요,10
4,16284557,싱싱하고 크기도 좋아요 근데 맛이 좀 덜들었네요좀난뒀다 먹어야겠어요,5,5,37,싱싱하고 크기도 좋아요 근데 맛이 좀 덜들었네요좀난뒀다 먹어야겠어요,37
...,...,...,...,...,...,...,...
1754921,8362913,최고포장 아 근데 진짜 많이,5,5,16,최고포장 아 근데 진짜 많이,16
1754922,21157725,진공포장에 아이스박스에 안전하게 배달잘 왔어요,5,5,25,진공포장에 아이스박스에 안전하게 배달잘 왔어요,25
1754923,56932594,좋아요 싸게 잘샀어요 좋아요,4,5,16,좋아요 싸게 잘샀어요 좋아요,16
1754924,29006409,제품 1개씩당 아이스팩을 같이묶어서 신선하게 잘받았네요,4,5,30,제품 1개씩당 아이스팩을 같이묶어서 신선하게 잘받았네요,30


In [87]:
word[word.length_clean == 0]

,memberSrl,content,prod_satis,ship_satis,length,clean,length_clean
33,47768417,ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ,1,1,12,,0
329,4540105,good!~~~~~~~,5,5,12,,0
514,47768417,ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ,1,1,12,,0
635,13015341,ㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇㅇ,5,5,11,,0
780,67182882,ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ,5,5,13,,0
...,...,...,...,...,...,...,...
1754614,10481813,ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ,5,5,10,,0
1754615,10481813,ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ,5,4,10,,0
1754616,10481813,ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ,5,5,10,,0
1754617,10481813,ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ,5,5,10,,0


In [21]:
word[word.length_clean == 1]

,memberSrl,content,prod_satis,ship_satis,length,clean,length_clean
929,20588797,gòooooooooooood,5,4,15,ò,1
1393,23750785,ㅎㅎㅎㅎㅎㅎㅎㅎ호ㅗㅗㅗ,4,4,12,호,1
1455,4285301,굿~~~~~~~~~~~,5,4,12,굿,1
2293,19026861,op;''/ jkioo,5,4,12,,1
2750,4285301,굿~~~~~~^^^^,4,4,11,굿,1
...,...,...,...,...,...,...,...
1751713,72854158,굿!!!!!!!!!!!,5,5,12,굿,1
1751714,72854158,굿!!!!!!!!!,4,5,10,굿,1
1752525,166367,굿.........,4,5,10,굿,1
1753434,20129841,굿.ㄱㅎㄱㄷㄱㅈㄱㅊㄱㅈㅋ,5,5,13,굿,1


In [22]:
word[word.length_clean == 2]

,memberSrl,content,prod_satis,ship_satis,length,clean,length_clean
109,48650957,굿굿!!!!!!!!,3,3,10,굿굿,2
129,48650957,굿굿!!!!!!!!!,4,4,11,굿굿,2
1681,3489193,ㄱㄹㅁ읏ㄹㄴ낙ㄱㅁㅎ,2,2,10,읏낙,2
1943,95357770,very delicious,5,5,15,,2
2516,76550186,별로..........,1,1,12,별로,2
...,...,...,...,...,...,...,...
1752598,25300581,만졷~~~~~~~~~,5,5,11,만졷,2
1753424,58954470,ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ좋음,5,5,14,좋음,2
1753951,1216654,ㅍㅎ호ㅠㅍㄴ먀ㅓㄱㄷㅌㅊㅎㅅ,4,4,14,호먀,2
1754218,1146869,0OLWAZOK0OLWAZOK,5,5,16,00,2


In [23]:
np.unique(word[word.length_clean == 3].clean)

array(['   ', '  3', '  6', '  7', '  ù', '  굿', '  머', '  흗', ' 55',
       ' 76', ' 9 ', ' 븍지', ' 실망', ' 쓰 ', ' 조앙', ' 존맛', ' 츄 ', '0제즈',
       '1  ', '12호', '2긔니', '2아요', '4”’', '5  ', '6듢든', '6튜푸', '7 8',
       '77퍼', '89 ', 'û55', 'ᆞᆞᆞ', 'ᆞ으응', 'Ỳ67', '⁷  ', '가가가', '가가니',
       '가격짱', '가뒧두', '가성비', '간쥑뉘', '감사 ', '감사링', '감사요', '갓순필', '강추요',
       '개꿀맛', '개존맛', '갡디늣', '갸어내', '고찮음', '곻고솦', '괘찮음', '괜찮길', '괜찮음',
       '괜춘요', '구 굿', '구8스', '구국국', '구숙궄', '구웃웃', '국굿굿', '굳  ', '굳 디',
       '굳 횻', '굳11', '굳굳 ', '굳굳굳', '굳밤 ', '굳짗짖', '굿  ', '굿굯근', '굿굿 ',
       '굿굿굿', '굿뜨요', '굿이요', '굿이쥬', '굿잡 ', '굿푸짐', '굿호호', '규규규', '그그긪',
       '그냥 ', '그냥요', '그니특', '극그극', '극긔극', '극니그', '극브븝', '극지짇', '근굿굿',
       '근너느', '근버는', '글그는', '급급긋', '급듯7', '긋  ', '긋긋 ', '긋니싲', '긔니그',
       '긘득그', '긪■∫', '기  ', '기기읟', '기너우', '기보너', '기보노', '기보니', '기보련',
       '기보보', '기본마', '깁낍핀', '깁읃빕', '꿀맛탱', '나서 ', '내내넌', '냄새가', '냐채텩',
       '너나마', '너난넌', '너눙5', '너마머', '너웅니', '넘맛 ', '넘작다', '네여 ', '노가가',
       '노녹4', '녹그시',

In [24]:
word[word.length_clean == 4]

,memberSrl,content,prod_satis,ship_satis,length,clean,length_clean
119,24355241,괜찮아요~~~~~~~~~,4,4,13,괜찮아요,4
157,31754761,좋아요 ♡♡♡♡♡♡♡♡♡,5,5,13,좋아요,4
167,9706537,짱이에요!!!!!!!!!,5,5,13,짱이에요,4
724,54441338,맛있어요~~~~~~~~~~~~~~~~~,5,5,21,맛있어요,4
904,8989901,그냥저냥~~~~~~~~~~,4,4,14,그냥저냥,4
...,...,...,...,...,...,...,...
1754200,61939258,맛있어요~~~~~~,5,5,10,맛있어요,4
1754513,53217437,맛있당 ㅎㅎㅎㅎㅎㅎㅎㅎ,5,5,12,맛있당,4
1754557,880028,맛이네유~~~~~~~~~,5,5,13,맛이네유,4
1754895,7684381,최고에용~~~~~~,5,5,10,최고에용,4


In [25]:
word[word.length_clean == 5]

,memberSrl,content,prod_satis,ship_satis,length,clean,length_clean
219,38139221,좋아요 굿!!!!!ㅎㅎ,5,5,12,좋아요 굿,5
662,72731354,양호합니다~~~~~~~~~,4,4,14,양호합니다,5
1049,37050277,만족합니다~~~!!,3,3,10,만족합니다,5
1064,13566158,감사드려요!!!!!!!!,5,5,13,감사드려요,5
1340,9996729,보통입니다.....,3,4,10,보통입니다,5
...,...,...,...,...,...,...,...
1754134,69133602,그냥그래요......,5,5,11,그냥그래요,5
1754631,43723973,잘받았어요~~~^^,3,3,10,잘받았어요,5
1754685,102385798,만족해요 !!!!!!!+!,5,5,14,만족해요,5
1754737,43723973,잘받았어요~~~^^,4,3,10,잘받았어요,5


In [26]:
word[word.length_clean == 6]

,memberSrl,content,prod_satis,ship_satis,length,clean,length_clean
62,64648098,잘 먹었어요~~~~~~~~~~~~~,4,4,19,잘 먹었어요,6
120,10205573,좋아요ㅎㅎ굳굳굳ㅎㅎ,5,5,10,좋아요굳굳굳,6
207,25050649,잘받았습니당ㅎㅎㅎㅎ,5,5,10,잘받았습니당,6
463,2123058,밤고구마네여ㅜㅜㅜㅜㅜㅜ,3,2,12,밤고구마네여,6
739,2334182,잘받았습니다★★★★,4,4,10,잘받았습니다,6
...,...,...,...,...,...,...,...
1754377,21838565,잘 받았어요~~~~~,5,5,11,잘 받았어요,6
1754392,33754689,쩰 맛있어요~~!~~~!,5,5,13,쩰 맛있어요,6
1754579,756029,잘 먹을게요~~~~~,5,5,11,잘 먹을게요,6
1754586,21838565,잘 받았어요!!!!!,5,5,11,잘 받았어요,6


# 상품 별점 보정하기
1. 유의미한 글자 길이가 15 이하일 때 5->4 , 1->2 이런 식으로 3차 함수 공식을 이용해서 보정
2. Tf-Idf를 기반으로 유저가 배송, 상품 둘 중에서 어떤 것을 중요시하는지 파악 -> 메모리 문제

In [13]:
def modify_under15(score):
    return (score-3)**3 / 8 + 3

In [14]:
word_under15 = word[word.length_clean < 15]
word_under15['prod_satis'] = word_under15.prod_satis.apply(modify_under15)
word_under15['ship_satis'] = word_under15.ship_satis.apply(modify_under15)

word_over15 = word[word.length_clean >= 15]
# word = pd.concat([word_under15, word[word.length_clean >= 15]])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
word_over15.clean

0                                   생각보다 조금 얇지만 맛있게 먹을게요
4                  싱싱하고 크기도 좋아요 근데 맛이 좀 덜들었네요좀난뒀다 먹어야겠어요
5                      며칠동안 놔뒀다가 먹으니까 되게 맛있네요 다음에 또 시킬게요
6                                        좀 실망스럽지만 잘 먹었어요
7                              고구마 해마다 여기서 먹어요 핵존맛 맛있습니다
                               ...                      
1754921                                 최고포장 아 근데 진짜 많이 
1754922                        진공포장에 아이스박스에 안전하게 배달잘 왔어요
1754923                                 좋아요 싸게 잘샀어요 좋아요 
1754924                   제품 1개씩당 아이스팩을 같이묶어서 신선하게 잘받았네요
1754925    오늘 받았어요 아직 먹진 않았지만 포장도 깔끔하고 고기 때깔도 좋네요 잘 먹을께요
Name: clean, Length: 867046, dtype: object

In [106]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# tf = TfidfVectorizer(smooth_idf=False, stop_words=None, sublinear_tf=True)
# x = tf.fit_transform(word_over15.clean)
# sk = pd.DataFrame(x.toarray())
# sk.columns = tf.get_feature_names()
# sk

# 실제 데이터에 보정된 별점 컬럼 추가하기
유의미한 글자 길이가 15 이하일 때 3차 함수 공식을 이용해서 별점 보정 후, 상품과 배송 만족도를 7:3으로 해서 satis 컬럼 생성

In [51]:
data = pd.read_json("/content/drive/My Drive/최강1조-추천 시스템/데이터 전처리/train_review_data.json")
# data = pd.read_json("/content/drive/My Drive/최강1조-추천 시스템/데이터 전처리/test_review_data.json")

In [52]:
def clean(x):
    text = re.sub('[-=_+,#\?;:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》☆★×♡♥·ᆢ\/]', '', x) # 특수문자 제거
    text = re.sub('[a-zA-Z]', '', text) # 영어 제거
    text = re.sub('[ㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣ]', '', text) # 모음 제거
    text = re.sub('[ㄱ-ㅎ]*', '', text) # 자음 반복 제거
    text = re.sub('[0-9]{3,}', '', text) # 숫자 3번 이상 나오면 제거
    return text

data['clean'] = data.content.apply(clean)
data['length_clean'] = data.clean.apply(len)

In [53]:
def modify_under15(score):
    return (score-3)**3 / 8 + 3

word_under15 = data[data.length_clean < 15]
word_under15['mod_prod_satis'] = word_under15.prod_satis.apply(modify_under15)
word_under15['mod_ship_satis'] = word_under15.ship_satis.apply(modify_under15)
word_under15['satis'] = word_under15.mod_prod_satis * 0.7 + word_under15.mod_ship_satis * 0.3
word_under15 = word_under15.drop('mod_prod_satis', axis=1).drop('mod_ship_satis', axis=1)

word_over15 = data[data.length_clean >= 15]
word_over15['satis'] = word_over15.prod_satis * 0.7 + word_over15.ship_satis * 0.3

word = pd.concat([word_under15, word_over15])
word = word.drop("clean", axis = 1).drop("length_clean", axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [54]:
# word.to_json("/content/drive/My Drive/최강1조-추천 시스템/데이터 전처리/train.json")
# word.to_json("/content/drive/My Drive/최강1조-추천 시스템/데이터 전처리/test.json")